
# Sistema de Venda de Ingressos - Parte 1

## O que é SQLite?

O **SQLite** é um banco de dados relacional **leve**, **portátil** e **sem servidor**. Ele salva os dados diretamente em um arquivo `.db`.

### Vantagens:
- Não requer instalação de servidor.
- Ideal para testes, protótipos e aplicações simples.
- Suporte nativo em Python.



## Criando o Banco e Tabelas com SQLite

```sql
CREATE TABLE evento (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    data TEXT NOT NULL,
    local TEXT NOT NULL
);

CREATE TABLE cliente (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE
);

CREATE TABLE ingresso (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cliente_id INTEGER,
    evento_id INTEGER,
    quantidade INTEGER,
    valor_unitario REAL,
    FOREIGN KEY (cliente_id) REFERENCES cliente(id),
    FOREIGN KEY (evento_id) REFERENCES evento(id)
);
```



# Parte 2 - Usando SQLite com Python

Vamos usar o módulo `sqlite3` do Python para acessar o banco, criar tabelas, inserir e consultar dados.


In [ ]:

import sqlite3

conn = sqlite3.connect('vendas.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS evento (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    data TEXT NOT NULL,
    local TEXT NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS cliente (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS ingresso (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cliente_id INTEGER,
    evento_id INTEGER,
    quantidade INTEGER,
    valor_unitario REAL,
    FOREIGN KEY (cliente_id) REFERENCES cliente(id),
    FOREIGN KEY (evento_id) REFERENCES evento(id)
)
''')

conn.commit()


In [ ]:

cursor.execute("INSERT INTO evento (nome, data, local) VALUES (?, ?, ?)", 
               ('Festival de Jazz', '2025-10-15', 'Centro de Convenções'))

cursor.execute("INSERT INTO cliente (nome, email) VALUES (?, ?)", 
               ('Maria Souza', 'maria@email.com'))

cursor.execute("INSERT INTO ingresso (cliente_id, evento_id, quantidade, valor_unitario) VALUES (?, ?, ?, ?)", 
               (1, 1, 3, 80.0))

conn.commit()


In [ ]:

cursor.execute("SELECT * FROM ingresso")
for row in cursor.fetchall():
    print(row)

conn.close()



# Parte 3 - SQLAlchemy com Arquitetura MVC Simplificada

Nesta parte, aplicamos a arquitetura **Model-View-Controller** com **SQLAlchemy**.


In [ ]:

from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()

class Evento(Base):
    __tablename__ = 'evento'
    id = Column(Integer, primary_key=True)
    nome = Column(String, nullable=False)
    data = Column(String, nullable=False)
    local = Column(String, nullable=False)

class Cliente(Base):
    __tablename__ = 'cliente'
    id = Column(Integer, primary_key=True)
    nome = Column(String, nullable=False)
    email = Column(String, nullable=False, unique=True)

class Ingresso(Base):
    __tablename__ = 'ingresso'
    id = Column(Integer, primary_key=True)
    cliente_id = Column(Integer, ForeignKey('cliente.id'))
    evento_id = Column(Integer, ForeignKey('evento.id'))
    quantidade = Column(Integer)
    valor_unitario = Column(Float)

    cliente = relationship("Cliente")
    evento = relationship("Evento")


In [ ]:

from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from modelos import Base, Evento, Cliente, Ingresso

engine = create_engine('sqlite:///vendas.db')
Session = sessionmaker(bind=engine)
session = Session()

def inicializar_banco():
    Base.metadata.create_all(engine)

def criar_evento(nome, data, local):
    evento = Evento(nome=nome, data=data, local=local)
    session.add(evento)
    session.commit()
    return evento

def criar_cliente(nome, email):
    cliente = Cliente(nome=nome, email=email)
    session.add(cliente)
    session.commit()
    return cliente

def registrar_venda(cliente_id, evento_id, quantidade, valor_unitario):
    ingresso = Ingresso(cliente_id=cliente_id, evento_id=evento_id,
                        quantidade=quantidade, valor_unitario=valor_unitario)
    session.add(ingresso)
    session.commit()

def listar_vendas():
    return session.query(Ingresso).all()


In [ ]:

from controller import inicializar_banco, criar_evento, criar_cliente, registrar_venda, listar_vendas

inicializar_banco()
evento = criar_evento("Festival de Jazz", "2025-10-01", "Arena Bahia")
cliente = criar_cliente("Joana Oliveira", "joana@email.com")
registrar_venda(cliente.id, evento.id, 2, 70.0)

for ingresso in listar_vendas():
    print(f"{ingresso.cliente.nome} comprou {ingresso.quantidade} ingresso(s) para '{ingresso.evento.nome}' por R$ {ingresso.valor_unitario:.2f}")
